# CS4661 (Intro to Data Science) - Shelter Animal Outcomes
# Author: Group #3
## Dataset: https://www.kaggle.com/c/shelter-animal-outcomes

In [1]:
"""
Preprocesses the shelter-animal-outcomes (sao) dataset

Parameters:

Returns:

"""

'\nPreprocesses the shelter-animal-outcomes (sao) dataset\n\nParameters:\n\nReturns:\n\n'

### Importing Libraries

In [2]:
import numpy as np
import pandas as pd

### Reading the sao dataset

In [3]:
df = pd.read_csv('../../data/train.csv') # stores the sao testing dataset 
df[:10] # displays first 10 rows

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
5,A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan
6,A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby
7,A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby
8,A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White
9,A677747,NaN,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White


In [4]:
feature_cols = list(df.columns) # stores each column name in a list
feature_cols # displays all column headers

['AnimalID',
 'Name',
 'DateTime',
 'OutcomeType',
 'OutcomeSubtype',
 'AnimalType',
 'SexuponOutcome',
 'AgeuponOutcome',
 'Breed',
 'Color']

In [5]:
junk_cols = [ 'AnimalID', 'Name', 'DateTime'] # useless columns
df = df.drop(junk_cols, axis=1) # drop the junk cols from the dataset
df[:10] # displays first 10 rows of changed df

,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
5,Transfer,Partner,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan
6,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby
7,Transfer,Partner,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby
8,Adoption,NaN,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White
9,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White


In [6]:

for x in range(len(df.Color)):
    df.Color[x] = df.Color[x].replace(' ','/')
    df.Color[x] = df.Color[x].replace('/', '|')

# pd.get_dummies(df["Color"])
# >>> Series(['a|b', 'a', 'a|c']).str.get_dummies()
df = pd.concat([df, df['Color'].str.get_dummies(sep='/')], axis=1)

df[:5]


,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Agouti,Apricot,Black,...,Smoke,Tabby,Tan,Tick,Tiger,Torbie,Tortie,Tricolor,White,Yellow
0,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown|White,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream|Tabby,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue|White,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue|Cream,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [7]:
colors = [c.replace(' ', '/').split('/') for c in list(df.Color)] # replace all spaces in Color with '/'
colors[:5] # displays first 5 rows of colors

[['Brown|White'], ['Cream|Tabby'], ['Blue|White'], ['Blue|Cream'], ['Tan']]

In [8]:
from collections import Counter

flat_colors = [c for clist in colors for c in clist] # [[color1], [color2], ..., [colorN]] > [color1, color2, ..., colorN]
colors_counts = Counter(flat_colors)
colors_counts # list the occurence of each color

Counter({'Brown|White': 884,
         'Cream|Tabby': 198,
         'Blue|White': 702,
         'Blue|Cream': 14,
         'Tan': 628,
         'Black|Tan': 672,
         'Blue|Tabby': 433,
         'Brown|Tabby': 1635,
         'Red|White': 331,
         'White': 931,
         'Black': 2292,
         'Silver': 17,
         'Brown': 639,
         'Black|Red': 17,
         'White|Cream': 36,
         'Orange|Tabby|White': 455,
         'Black|White': 2824,
         'Brown|Brindle|White': 450,
         'Black|Brown': 436,
         'Orange|Tabby': 841,
         'Chocolate|White': 224,
         'White|Tan': 389,
         'Cream|Tabby|White': 79,
         'Blue': 450,
         'Calico': 517,
         'Torbie': 335,
         'Brown|Black': 333,
         'Yellow': 185,
         'Tricolor': 752,
         'White|Black': 643,
         'Tortie': 530,
         'Blue|Tabby|White': 241,
         'Gray|White': 96,
         'Tan|Black': 183,
         'Tan|White': 773,
         'Buff': 199,
         'Br

In [9]:
list(colors_counts.keys())

['Brown|White',
 'Cream|Tabby',
 'Blue|White',
 'Blue|Cream',
 'Tan',
 'Black|Tan',
 'Blue|Tabby',
 'Brown|Tabby',
 'Red|White',
 'White',
 'Black',
 'Silver',
 'Brown',
 'Black|Red',
 'White|Cream',
 'Orange|Tabby|White',
 'Black|White',
 'Brown|Brindle|White',
 'Black|Brown',
 'Orange|Tabby',
 'Chocolate|White',
 'White|Tan',
 'Cream|Tabby|White',
 'Blue',
 'Calico',
 'Torbie',
 'Brown|Black',
 'Yellow',
 'Tricolor',
 'White|Black',
 'Tortie',
 'Blue|Tabby|White',
 'Gray|White',
 'Tan|Black',
 'Tan|White',
 'Buff',
 'Brown|Tabby|White',
 'Red',
 'Blue|Tan',
 'Seal|Point',
 'Brown|Brindle',
 'White|Brown',
 'Gray',
 'Yellow|Brindle|White',
 'Fawn|White',
 'Flame|Point',
 'Black|Tabby|White',
 'Tortie|Calico',
 'Black|Brown|Brindle',
 'White|Gray',
 'Tan|Silver',
 'Red|Tick|Black',
 'White|Cream|Tabby',
 'Blue|Merle',
 'Chocolate|Tan',
 'Sable',
 'Brown|Merle|White',
 'Brown|Tiger|White',
 'Liver|White',
 'White|Blue|Tabby',
 'Black|Gray',
 'Cream',
 'Chocolate',
 'Blue|Point',
 'Apric